In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

# Load data

In [ ]:
train_path = "../../data/Santander/train_ver2.csv"

target_columns = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
other_columns = ['fecha_dato', 'ncodpers'] 
all_columns = target_columns + other_columns

dtypes_columns = {}
for c in target_columns:
    dtypes_columns[c] = np.float16

#limit_rows = 50
limit_rows = 50000000

df_train = pd.read_csv(train_path, usecols=all_columns, dtype=dtypes_columns, nrows=limit_rows)
#df_train = pd.read_csv(train_path, usecols=all_columns,  nrows=limit_rows)

# Data preprocessing and preparation

In [ ]:
#for now, lets work only with a few ncodpers removing most of them
all_codpers = df_train['ncodpers'].unique()
print(len(all_codpers))
use_codpers = all_codpers[0:5000]
df_train = df_train[df_train.ncodpers.isin(use_codpers)]
print(len(df_train['ncodpers'].unique()))

In [ ]:
#for c in df_train.columns:
    #print(c + ': ' + str(df_train[c].isnull().sum()))
df_train['ind_nomina_ult1'].fillna(0, inplace=True)
df_train['ind_nom_pens_ult1'].fillna(0, inplace=True)
df_test = df_train[df_train['fecha_dato'] == '2016-05-28']
df_train = df_train[df_train['fecha_dato'] != '2016-05-28']
print(df_train['fecha_dato'].value_counts())
print(df_test['fecha_dato'].value_counts())

In [ ]:
print(len(df_train))
print(len(df_test))

In [ ]:
n_products = len(target_columns)

In [ ]:
X_train = []
Y_train = []
grouped = df_train.groupby('ncodpers')
seq_length = 5 # timesteps
for ncodpers,group in grouped:
    j = j + 1
    for i in range(len(group) - seq_length):
        x = group[target_columns].values[i:(i+seq_length),:]
        y = group[target_columns].values[i+seq_length,:]
        X_train.append(x)
        Y_train.append(y)


In [ ]:

cmp = list(Y_train[0] == df_train[df_train.ncodpers == 15889][target_columns].iloc[seq_length].values)
assert (True in cmp)
cmp = list(Y_train[10] == df_train[df_train.ncodpers == 15889][target_columns].iloc[-1].values)
assert (True in cmp)
#all(item == True for item in list(cmp))
#list(Y_train[0] == df_train[df_train.ncodpers == 15889][target_columns].iloc[seq_length].values)

In [ ]:
X_test = []
Y_test = []
grouped = df_test.groupby('ncodpers')
seq_length = 5 # timesteps
for ncodpers,group in grouped:
    for i in range(len(group) - seq_length):
        x = group[target_columns].values[i:(i+seq_length),:]
        y = group[target_columns].values[i+seq_length,:]
        X_test.append(x)
        Y_test.append(y)

# TensorFlow Model 1

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np


In [ ]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = n_products # data input
n_hidden = 64 # hidden layer num of features
n_output = n_products
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, seq_length, n_input])
y = tf.placeholder("float", [None, n_output])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    
    x = tf.transpose(x, [1, 0, 2])
    
    
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    

    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    #x = tf.split(x, n_steps, 0)
    x = tf.split(split_dim=0, num_split=seq_length, value=x)


    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    

    # Get lstm cell output
    #tf.nn.rnn(cell, inputs,#initial_state=self._initial_state)
    #outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    outputs, states = tf.nn.rnn(lstm_cell, x, dtype=tf.float32)


    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)




# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
#correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()


In [ ]:

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_x, y: batch_y})
        
        # Write logs at every iteration
        summary_writer.add_summary(summary, step)
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + 
                  "{:.6f}".format(loss) + ", Training Accuracy= " + 
                  "{:.5f}".format(acc))
            
            
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", 
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))